In [11]:
import sys, os, re
import pandas as pd
import numpy as np
import json
import requests
import csv


In [12]:
string_name = pd.read_csv('data/9606.protein.info.v11.5.txt', sep="\t")

In [13]:
string_name.head()

,#string_protein_id,preferred_name,protein_size,annotation
0,9606.ENSP00000000233,ARF5,180,ADP-ribosylation factor 5; GTP-binding protein...
1,9606.ENSP00000000412,M6PR,277,Cation-dependent mannose-6-phosphate receptor;...
2,9606.ENSP00000001008,FKBP4,459,Peptidyl-prolyl cis-trans isomerase FKBP4; Imm...
3,9606.ENSP00000001146,CYP26B1,512,Cytochrome P450 26B1; Involved in the metaboli...
4,9606.ENSP00000002125,NDUFAF7,441,"Protein arginine methyltransferase NDUFAF7, mi..."


In [14]:
STRING_df_dict = {}

string_name_dict = string_name.to_dict('split')

for i in range(len(string_name_dict['data'])):
    STRING_df_dict[string_name_dict['data'][i][0]] = {"name": string_name_dict['data'][i][1], "description": string_name_dict['data'][i][3].split(";")[0].strip()}

del string_name, string_name_dict

In [15]:
##string

string_link = pd.read_csv('data/9606.protein.links.detailed.v11.5.txt', sep=" ")

string_link.head()

,protein1,protein2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score
0,9606.ENSP00000000233,9606.ENSP00000379496,0,0,0,54,0,0,144,155
1,9606.ENSP00000000233,9606.ENSP00000314067,0,0,0,0,180,0,61,197
2,9606.ENSP00000000233,9606.ENSP00000263116,0,0,0,62,152,0,101,222
3,9606.ENSP00000000233,9606.ENSP00000361263,0,0,0,0,161,0,64,181
4,9606.ENSP00000000233,9606.ENSP00000409666,0,0,0,82,213,0,72,270


In [16]:
combine_cutoff_score = 900

string_link_dict = string_link[string_link["combined_score"] >= combine_cutoff_score].to_dict('split')

link_set = set()

STRING_to_add = {}

for i in range(len(string_link_dict['data'])):
    protein_1 = string_link_dict['data'][i][0]
    protein_2 = string_link_dict['data'][i][1]

    #print (protein_1, protein_2)

    if "name" in STRING_df_dict[protein_1] and "name" in STRING_df_dict[protein_2]:
        protein_name_1 = STRING_df_dict[protein_1]["name"]
        protein_name_2 = STRING_df_dict[protein_2]["name"]

        t = tuple(sorted([protein_name_1, protein_name_2]))
        link_set.add(t)

        STRING_to_add[protein_name_1.lower()] = protein_name_1
        STRING_to_add[protein_name_2.lower()] = protein_name_2
    else:
        print ("No name for protein: ", protein_1, protein_2)
del string_link_dict, string_link

In [17]:
len(link_set)

123577

In [18]:
len(STRING_to_add)

11813

Now correct the names in hetionet.gene to STRING

In [19]:
json_connections = set()


with open('data/hetionet-v1.0.json', 'r') as f:
    json_data = json.load(f)

    id_name = {}

    nodes = []
    for g in json_data["nodes"]:
        if g["kind"] == "Gene":
            if g["name"].lower() in STRING_to_add:
                g["name"] = STRING_to_add[g["name"].lower()]

        id_name[g["identifier"]] = g["name"]
        
        if "data" in g:
            del g["data"]
        nodes.append(g)

    with open('data/hetionet_string_corrected_gene_name_nodes.json', 'w') as outfile:
        json.dump(nodes, outfile, indent=2)

    edges = []
    for edge in json_data["edges"]:
        id_1 = edge["source_id"][1]
        name_1 = id_name[id_1]
        edge["source_id"][1] = name_1
        
        id_2 = edge["target_id"][1]
        name_2 = id_name[id_2]
        edge["target_id"][1] = name_2

        if "data" in edge:
            del edge["data"]
    
        edges.append(edge)

    with open('data/hetionet_string_corrected_gene_name_edges.json', 'w') as outfile:
        json.dump(edges, outfile, indent=2)

Now correct the names with kegg

In [20]:
df = pd.read_csv("data/disease.csv")

string_name_dict = df.to_dict('split')

kegg_disease_name = {}

for i in range(len(string_name_dict['data'])):
    kegg_disease_name[string_name_dict['data'][i][1].lower()] = string_name_dict['data'][i][1]


df = pd.read_csv("data/drug.csv")

string_name_dict = df.to_dict('split')

kegg_compound_name = {}

for i in range(len(string_name_dict['data'])):
    kegg_compound_name[string_name_dict['data'][i][1].lower()] = string_name_dict['data'][i][1]


df = pd.read_csv("data/gene_tmp.csv")

string_name_dict = df.to_dict('split')

kegg_gene_name = {}

for i in range(len(string_name_dict['data'])):
    kegg_gene_name[string_name_dict['data'][i][1].lower()] = string_name_dict['data'][i][1]




with open('data/hetionet_string_corrected_gene_name_nodes.json', 'r') as f:
    nodes = json.load(f)

    new_nodes = []
    for node in nodes:
        if node["kind"] == "Disease":
            if node["name"].lower() in kegg_disease_name:
                node["name"] = kegg_disease_name[node["name"].lower()]
        elif node["kind"] == "Compound":
            if node["name"].lower() in kegg_compound_name:
                node["name"] = kegg_compound_name[node["name"].lower()]
        elif node["kind"] == "Gene":
            if node["name"].lower() in kegg_gene_name:
                node["name"] = kegg_gene_name[node["name"].lower()]
        
        new_nodes.append(node)

        
    with open('data/hetionet_kegg_corrected_gene_name_nodes.json', 'w') as outfile:
        json.dump(new_nodes, outfile, indent=2)


with open('data/hetionet_string_corrected_gene_name_edges.json', 'r') as f:
    edges = json.load(f)

    new_edges = []
    for edge in edges:
        if edge["source_id"][0] == "Disease":
            if edge["source_id"][1].lower() in kegg_disease_name:
                edge["source_id"][1] = kegg_disease_name[edge["source_id"][1].lower()]

        if edge["target_id"][0] == "Disease":
            if edge["target_id"][1].lower() in kegg_disease_name:
                edge["target_id"][1] = kegg_disease_name[edge["target_id"][1].lower()]
        

        if edge["source_id"][0] == "Compound":
            if edge["source_id"][1].lower() in kegg_compound_name:
                edge["source_id"][1] = kegg_compound_name[edge["source_id"][1].lower()]

        if edge["target_id"][0] == "Compound":
            if edge["target_id"][1].lower() in kegg_compound_name:
                edge["target_id"][1] = kegg_compound_name[edge["target_id"][1].lower()]

        
        if edge["source_id"][0] == "Gene":
            if edge["source_id"][1].lower() in kegg_gene_name:
                edge["source_id"][1] = kegg_gene_name[edge["source_id"][1].lower()]

        if edge["target_id"][0] == "Gene":
            if edge["target_id"][1].lower() in kegg_gene_name:
                edge["target_id"][1] = kegg_gene_name[edge["target_id"][1].lower()]
        
        new_edges.append(edge)

        
    with open('data/hetionet_kegg_corrected_gene_name_edges.json', 'w') as outfile:
        json.dump(new_edges, outfile, indent=2)

save string links and nodes

In [21]:
STRING_to_add.values()
string_name = pd.read_csv('data/9606.protein.info.v11.5.txt', sep="\t")

print (len(string_name))
#df_kegg_disease_diff_hetionet = df_kegg_disease[~df_kegg_disease["name"].str.lower().isin(hetionet_disease_name)]
#print (string_name.head())

string_name = string_name[string_name["preferred_name"].isin(STRING_to_add.values())]
string_name["description"] = string_name["annotation"].str.split(";").str[0]

string_name.drop(["#string_protein_id", "protein_size", "annotation"], axis = 1, inplace = True)

string_name.rename({'preferred_name': 'name'}, axis='columns', inplace=True)

print (string_name.head())

print (len(string_name))

string_name.to_csv("data/string_to_add_nodes.csv", index = False, quoting=csv.QUOTE_ALL)


19566
      name                                        description
0     ARF5                          ADP-ribosylation factor 5
1     M6PR      Cation-dependent mannose-6-phosphate receptor
2    FKBP4          Peptidyl-prolyl cis-trans isomerase FKBP4
3  CYP26B1                               Cytochrome P450 26B1
4  NDUFAF7  Protein arginine methyltransferase NDUFAF7, mi...
11814


In [22]:
content = []
"""
{
    "source_id": [
      "Gene",
      "CALM1"
    ],
    "target_id": [
      "Gene",
      "VHL"
    ],
    "kind": "interacts",
    "direction": "both"
  },
"""


for new_edge in link_set:
    content.append({"source_id": ["Gene", new_edge[0]], "target_id": ["Gene", new_edge[1]], "kind": "STRING"})

with open('data/string_to_add_edges.json', 'w') as outfile:
    json.dump(content, outfile, indent=2)